In [1]:
from dotenv import load_dotenv
load_dotenv("/home/marshath/play/chainlink/algovate/.env")

Python-dotenv could not parse statement starting at line 4


True

In [2]:
import pickle
import tiktoken
import pandas as pd
import ipywidgets as widgets
from typing import List, Any, Dict
from IPython.display import clear_output
from IPython.display import display, Markdown
from langchain.docstore.document import Document
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.retrievers import TFIDFRetriever
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

from pydantic import BaseModel
from langchain.schema import BaseRetriever, Document

In [3]:
def retrieve_docs(df, retriever):
    # Add retrieved_docs column if it doesn't exist
    df['retrieved_docs'] = df.get('retrieved_docs', '')

    for index, row in df.iterrows():
        retrieved_docs = retriever.get_relevant_documents(row['question'])

        docs = ""
        for d in retrieved_docs:
            if docs == "":
                docs = d.page_content
            else:
                docs = docs + "\n\n:::NEXT DOC:::\n\n" + d.page_content

        # Store retrieved docs in the DataFrame
        df.at[index, 'retrieved_docs'] = docs

In [4]:
with open("/home/marshath/play/chainlink/algovate/algovate/data/combined_documents.pkl", "rb") as f:
    documents = pickle.load(f)

In [5]:
enc = tiktoken.get_encoding("cl100k_base")
for doc in documents:
    num_tokens = len(enc.encode(doc.page_content))
    if num_tokens > 6000:
        print(num_tokens)

6409
7912
19252
40097
14632


In [6]:
class CustomeSplitter:
    def __init__(self, chunk_threshold=6000, chunk_size=6000, chunk_overlap=50):
        self.chunk_threshold = chunk_threshold
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.enc = tiktoken.get_encoding("cl100k_base")
        self.splitter = TokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    def token_counter(self, document):
        tokens = self.enc.encode(document.page_content)
        return len(tokens)

    def split(self, documents):
        chunked_documents = []
        for i, doc in enumerate(documents):
            try:
                if self.token_counter(doc) > self.chunk_threshold:
                    chunks = self.splitter.split_documents([doc])
                    chunks= [Document(page_content=chunk.page_content, metadata={"source": f"{chunk.metadata['source']} chunk {i}"}) for i, chunk in enumerate(chunks)]
                    chunked_documents.extend(chunks)
                else:
                    chunked_documents.append(doc)
            except:
                chunked_documents.append(doc)
                print(f"Error on document {i}")
                print(doc)

        return chunked_documents

In [7]:
splitter = CustomeSplitter()
chunked_documents = splitter.split(documents)

In [8]:
class CustomRetriever(BaseRetriever, BaseModel):
    full_docs: List[Document]
    base_retriever:BaseRetriever = None

    class Config:
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True

    @classmethod
    def from_documents(
        cls,
        full_docs: List[Document],
        search_kwargs: Dict[str, Any] = {},
        **kwargs: Any,
    ):
        splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)
        split_docs = splitter.split_documents(documents)
        vector_store = FAISS.from_documents(split_docs, embedding=OpenAIEmbeddings())

        return cls(full_docs=full_docs, base_retriever=vector_store.as_retriever(**search_kwargs), **kwargs)

    def get_relevant_documents(self, query: str) -> List[Document]:  
        results =  self.base_retriever.get_relevant_documents(query=query)
        doc_ids = [doc.metadata["source"] for doc in results]
        full_retrieved_docs = [d for d in chunked_documents if d.metadata["source"] in doc_ids]
        return full_retrieved_docs
        
    async def aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError

In [9]:
retriever = CustomRetriever.from_documents(chunked_documents, search_kwargs={"k": 1})
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.)
chain = RetrievalQAWithSourcesChain.from_chain_type(retriever=retriever, llm=llm, chain_type="stuff")

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 970d91aa9f88825b922f210256a6ed9e in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 970d91aa9f88825b922f210256a6ed9e in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 970d91aa9f88

In [10]:
df = pd.read_csv("/home/marshath/play/chainlink/algovate/algovate/data/auto_eval_pairs_v3.csv")

In [11]:
# retriever = TFIDFRetriever.from_documents(chunked_documents, k=2)
# llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.)
# chain = RetrievalQAWithSourcesChain.from_chain_type(retriever=retriever, llm=llm, chain_type="stuff")

### Custom retriever

In [12]:
df2 = df.copy()
for i, gt in df2.iterrows():
    try:
        res = chain(gt["question"])
        df2.loc[i, "result"] = res["answer"]
        df2.loc[i, "sources"] = res["sources"]
    except Exception as e:
        print(e)
        print(f"Error on {i}")
        print(gt["question"])
    print(f"Done with {i}")

Done with 0
Done with 1
Done with 2
Done with 3
Done with 4
Done with 5
Done with 6
Done with 7
Done with 8
Done with 9
Done with 10
Done with 11
Done with 12
Done with 13
Done with 14
Done with 15
Done with 16
Done with 17
Done with 18
Done with 19
Done with 20
Done with 21
Done with 22
Done with 23
Done with 24
Done with 25
Done with 26
Done with 27
Done with 28
Done with 29
Done with 30
Done with 31


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 39730e85512d17397def9acdca023173 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 39730e85512d17397def9acdca023173 in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 39730e85512d

Done with 32
Done with 33
Done with 34
Done with 35
Done with 36
Done with 37
Done with 38
Done with 39
Done with 40
Done with 41
Done with 42
Done with 43
Done with 44
This model's maximum context length is 16385 tokens. However, your messages resulted in 21437 tokens. Please reduce the length of the messages.
Error on 45
What is the purpose of the `functions-simulate` command in the Chainlink Functions hardhat starter kit?
Done with 45
Done with 46
Done with 47
Done with 48
Done with 49
Done with 50
Done with 51
Done with 52
Done with 53
Done with 54
Done with 55
Done with 56
Done with 57
Done with 58
Done with 59
Done with 60
Done with 61
Done with 62
Done with 63
Done with 64
Done with 65
Done with 66
Done with 67
Done with 68
Done with 69
Done with 70
Done with 71


In [13]:
retrieve_docs(df2, retriever)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID c08a07a7d306cd840b15cc2d1f69319e in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID c08a07a7d306cd840b15cc2d1f69319e in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID c08a07a7d306

In [14]:
df2.to_csv("/home/marshath/play/chainlink/algovate/test_chainlink/autoq_code1_16kmodel.csv", index=False)

### Richard's Question

In [7]:
with open("/home/marshath/play/chainlink/algovate/algovate/data/ground_truths.pkl", "rb") as f:
    ground_truths = pickle.load(f)

df = pd.DataFrame(ground_truths)

In [10]:
df2 = df.copy()
for i, gt in df2.iterrows():
    try:
        res = chain(gt["question"])
        df2.loc[i, "result"] = res["answer"]
        df2.loc[i, "sources"] = res["sources"]
    except Exception as e:
        print(e)
        print(f"Error on {i}")
        print(gt["question"])

    print(f"Done with {i}")

In [16]:
evaluate_dataframe_answer(df2)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='partial', style=ButtonStyle())

In [18]:
df2.to_csv("/home/marshath/play/chainlink/algovate/test_chainlink/16kmodel.csv", index=False)

### AutoQ Questions

In [8]:
df = pd.read_csv("/home/marshath/play/chainlink/algovate/algovate/data/auto_eval_pairs_v2.csv")

In [9]:
df3 = df.copy()
for i, gt in df3.iterrows():
    try:
        res = chain(gt["question"])
        df3.loc[i, "result"] = res["answer"]
        df3.loc[i, "sources"] = res["sources"]
    except Exception as e:
        print(e)
        print(f"Error on {i}")
        print(gt["question"])

    print(f"Done with {i}")

Done with 0
Done with 1
Done with 2
Done with 3
Done with 4
Done with 5
Done with 6
Done with 7
Done with 8
Done with 9
Done with 10
Done with 11
Done with 12
Done with 13
Done with 14
Done with 15
Done with 16
Done with 17
Done with 18
Done with 19
Done with 20
Done with 21
Done with 22
Done with 23
Done with 24
Done with 25
Done with 26
Done with 27
Done with 28
Done with 29
Done with 30
Done with 31
Done with 32
Done with 33
Done with 34
Done with 35
Done with 36
Done with 37
Done with 38
Done with 39
Done with 40
Done with 41
Done with 42
Done with 43
Done with 44
Done with 45
Done with 46
Done with 47
Done with 48
Done with 49
Done with 50
Done with 51
Done with 52
Done with 53
Done with 54
Done with 55
Done with 56
Done with 57
Done with 58
Done with 59


In [10]:
def retrieve_docs(df, retriever):
    # Add retrieved_docs column if it doesn't exist
    df['retrieved_docs'] = df.get('retrieved_docs', '')

    for index, row in df.iterrows():
        retrieved_docs = retriever.get_relevant_documents(row['question'])

        docs = ""
        for d in retrieved_docs:
            if docs == "":
                docs = d.page_content
            else:
                docs = docs + "\n\n:::NEXT DOC:::\n\n" + d.page_content

        # Store retrieved docs in the DataFrame
        df.at[index, 'retrieved_docs'] = docs

In [11]:
retrieve_docs(df3, retriever)

In [17]:
df3.to_csv("/home/marshath/play/chainlink/algovate/test_chainlink/autoq_combineddoc_16kmodel_v2.csv", index=False)